## Layer height estimation

First, let's import the necessary functions

In [ ]:
import os
from time import time as current_time
from copy import deepcopy

import numpy as np
import scipy.ndimage as nd
import pandas as pd
import matplotlib.pyplot as plt

import panel
import vtk

from timagetk.components import SpatialImage, LabelledImage
from timagetk.io import imread, imsave

from cellcomplex.utils.array_dict import array_dict

from cellcomplex.property_topomesh.creation import vertex_topomesh
from cellcomplex.property_topomesh.analysis import compute_topomesh_property
from cellcomplex.property_topomesh.morphology import topomesh_binary_property_morphological_operation, topomesh_binary_property_fill_holes
from cellcomplex.property_topomesh.extraction import property_filtering_sub_topomesh, topomesh_connected_components, clean_topomesh

from cellcomplex.property_topomesh.visualization.vtk_actor_topomesh import VtkActorTopomesh

from tissue_nukem_3d.utils.matplotlib_tools import draw_box

from tissue_analysis.tissue_analysis import TissueAnalysis, TissueImage

from visu_core.matplotlib import glasbey
from visu_core.matplotlib.colormap import plain_colormap

from visu_core.vtk.display import vtk_display_notebook, vtk_save_screenshot_actors
from visu_core.vtk.volume import vtk_image_volume
from visu_core.vtk.actor import vtk_actor

from visu_core.vtk.utils.image_tools import vtk_image_data_from_image, image_to_vtk_cell_polydatas, image_to_surface_intensity_polydata, image_to_surface_cell_scalar_property_polydata
from visu_core.vtk.utils.polydata_tools import vtk_combine_polydatas, vtk_tube_polydata, vtk_slice_polydata


### Import locally defined tools

In [ ]:
from utils.tissue_analysis_tools import compute_tissue_cell_centers, compute_tissue_cell_volumes, compute_tissue_cell_surface_centers
from utils.tissue_analysis_tools import compute_tissue_cell_neighbors_from_surfels, compute_tissue_cell_layer_surface_centers
from utils.tissue_analysis_tools import compute_tissue_cell_layer_from_surface_mesh, compute_tissue_cell_heights
from utils.tissue_analysis_tools import compute_tissue_cell_property_from_surface_mesh, compute_tissue_cell_property_from_layer_surface_meshes
from utils.tissue_analysis_tools import compute_tissue_cell_binary_property_from_surface_mesh, tissue_cell_binary_property_connected_components
from utils.tissue_analysis_tools import tissue_cell_binary_property_morphological_operation, tissue_cell_binary_property_layer_propagation
from utils.tissue_analysis_tools import tissue_analysis_to_dataframe
from utils.tissue_mesh_tools import segmentation_surface_topomesh, tissue_surface_topomesh, tissue_layer_surface_meshes

### File path information

In [ ]:
dirname = '/Users/gcerutti/Projects/RDP/LayerHeight_Marketa/test/'
#dirname = '/home/carlos/Documents/WORK/Image analysis/20160625 MS-E3 CLV-DR5/LD/RAW'

#filename = 'E15_SAM02'
filename = 'E_set_blue_SAM02'
#filename = 'Col-0-PI-SAM3'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM3'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM5'

### Algorithm parameters

In [ ]:
# image parameters
channel_names = ['PI']
#channel_names = ['Signal','PI']
#channel_names = ['CLV3','DR5','PI']
membrane_channel = 'PI'
#signal_channels = ['CLV3','DR5']
#signal_channels = ['Signal']
signal_channels = []
microscope_orientation = 1

### Reading the segmented image

In [ ]:
segmentation_filename = dirname + '/' + filename + '/' + filename + '_' + membrane_channel + '_seg.tif'

seg_img = imread(segmentation_filename)
seg_img = LabelledImage(seg_img, no_label_id=0)

tissue = TissueAnalysis(TissueImage(seg_img, background=1),auto_init=False)

In [ ]:
seg_image_data = vtk_image_data_from_image(seg_img.get_array()%256,voxelsize=seg_img.voxelsize)
seg_img_volume = vtk_image_volume(seg_image_data,alpha_mode='label',background_label=1,
                                  colormap='glasbey',value_range=(0,255),opacity=1)

vtk_save_screenshot_actors([seg_img_volume],dirname + '/' + filename + '/' + filename + "_segmentation.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))

### Computing cell neighbors

In [ ]:
compute_tissue_cell_neighbors_from_surfels(tissue)

In [ ]:
actors = []

seg_image_data = vtk_image_data_from_image(seg_img.get_array()%256,voxelsize=seg_img.voxelsize)
no_seg_img_volume = vtk_image_volume(seg_image_data ,opacity=0.)
actors += [no_seg_img_volume]

labels = [l for l in seg_img.labels() if l!=1]

cell_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                             labels=labels,
                                             subsampling=[int(np.round(0.8/v)) for v in seg_img.voxelsize],
                                             smoothing=10)
cell_polydata = vtk_combine_polydatas([cell_polydatas[l] for l in labels])
cell_actor = vtk_actor(cell_polydata,colormap='glasbey',value_range=(0,255))
actors += [cell_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_polydatas.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
vtk_display_notebook(actors)

### Extracting meristem surface

In [ ]:
surface_topomesh = segmentation_surface_topomesh(seg_img, microscope_orientation=microscope_orientation, compute_curvature=True)

In [ ]:
actors = []

actors += [no_seg_img_volume]

surface_actor = VtkActorTopomesh(surface_topomesh,2)
surface_actor.update(colormap=plain_colormap('w'))
actors += [surface_actor]

surface_edge_actor = VtkActorTopomesh(surface_topomesh, 1, line_glyph='tube', glyph_scale=0.2)
surface_edge_actor.update(colormap=plain_colormap('k'), opacity=0.2)
actors += [surface_edge_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_mesh.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))

In [ ]:
actors = []

seg_image_data = vtk_image_data_from_image(seg_img.get_array()%256,voxelsize=seg_img.voxelsize)
no_seg_img_volume = vtk_image_volume(seg_image_data ,opacity=0.)
actors += [no_seg_img_volume]

surface_actor = VtkActorTopomesh(surface_topomesh,2,property_name='mean_curvature',property_degree=0)
surface_actor.update(colormap='RdBu_r',value_range=(-5e-2,5e-2))
actors += [surface_actor]

normal_actor = VtkActorTopomesh(surface_topomesh,0,property_name='normal',glyph_scale=3.)
normal_actor.update(colormap=plain_colormap('chartreuse'))
actors += [normal_actor]

surface_edge_actor = VtkActorTopomesh(surface_topomesh, 1, line_glyph='tube', glyph_scale=0.2)
surface_edge_actor.update(colormap=plain_colormap('k'),opacity=0.2)
actors += [surface_edge_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_mesh_curvature_normals.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
vtk_display_notebook(actors)

### Mapping cells on SAM surface

In [ ]:
cell_labels = np.array([l for l in tissue.get_label_ids()])

compute_tissue_cell_centers(tissue)
cell_center = array_dict(tissue.label.get_property('center'))

compute_tissue_cell_volumes(tissue)
cell_volumes = array_dict(tissue.label.get_property('volume'))

compute_tissue_cell_surface_centers(tissue)
cell_surface_center = array_dict(tissue.label.get_property('surface_center'))

surface_cells = cell_surface_center.keys()


In [ ]:
surface_topomesh = tissue_surface_topomesh(tissue, surface_topomesh)
surface_vertex_cell = surface_topomesh.wisp_property('cell',0).values(list(surface_topomesh.wisps(0)))

In [ ]:
actors = []

seg_image_data = vtk_image_data_from_image(seg_img.get_array()%256,voxelsize=seg_img.voxelsize)
no_seg_img_volume = vtk_image_volume(seg_image_data ,opacity=0.)
actors += [no_seg_img_volume]

surface_actor = VtkActorTopomesh(surface_topomesh,2,property_name='label',property_degree=0)
surface_actor.update(colormap='glasbey',value_range=(0,255))
actors += [surface_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_mesh_cell.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
vtk_display_notebook(actors)

### Estimating cell layer using surface

In [ ]:
compute_tissue_cell_layer_from_surface_mesh(tissue, surface_topomesh)
cell_layer = array_dict(tissue.label.get_property('layer'))

compute_tissue_cell_layer_surface_centers(tissue)

In [ ]:
layer_cells = [c for c in cell_labels
                if cell_center[c][1] > 0.48*seg_img.extent[1]]

actors = []

layer_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                              labels=layer_cells,
                                              cell_property=tissue.label.get_property('layer'),
                                              cell_polydatas=cell_polydatas)
layer_polydata = vtk_combine_polydatas([layer_polydatas[l] for l in layer_cells])
layer_actor = vtk_actor(layer_polydata,colormap='jet',value_range=(0,4))
actors += [layer_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_layer_polydatas.png",focal_point=(0,1,0),view_up=(0,0,microscope_orientation),size=(2000,2000))
vtk_display_notebook(actors)

### Extracting inter-layer surfaces

In [ ]:
layer_surface_meshes = tissue_layer_surface_meshes(tissue, surface_topomesh=surface_topomesh, microscope_orientation=microscope_orientation)

In [ ]:
actors = []

for layer in [1,2,3]:
    layer_cells = [c for c in cell_labels if cell_layer[c]==layer]

    actor = VtkActorTopomesh(layer_surface_meshes[layer],2,property_name='label',property_degree=0)
    actor.update_polydata()
    # actor.polydata = vtk_slice_polydata(vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
    # actor.polydata = vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=10.)
    actor.display_polydata = vtk_slice_polydata(actor.polydata,axis='x',
                                                position=(layer+1)*seg_img.extent[0]/10.,
                                                width=(layer+1)*seg_img.extent[0]/5.)
    actor.update_actor(colormap='glasbey', value_range=(0, 255))
    actors += [actor]

    actors += [no_seg_img_volume]

    #vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_L"+str(layer)+"_mesh_cell.png", focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_inter_layer_surfaces.png", focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
#vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_inter_layer_surfaces.png",focal_point=(1,0,0),view_up=(0,0,microscope_orienation),size=(2000,2000))
vtk_display_notebook(actors)

### Estimating cell normal vectors

In [ ]:
compute_tissue_cell_property_from_layer_surface_meshes(tissue, 'normal', layer_surface_meshes)
cell_layer_surface_center = tissue.label.get_property('layer_surface_center')
cell_layer_surface_normals = tissue.label.get_property('normal')

cell_vertex_topomesh = vertex_topomesh(cell_layer_surface_center)
cell_vertex_topomesh.update_wisp_property('label',0,{c:c%256 for c in cell_vertex_topomesh.wisps(0)})
cell_vertex_topomesh.update_wisp_property('layer',0,{c:cell_layer[c] for c in cell_vertex_topomesh.wisps(0)})
cell_vertex_topomesh.update_wisp_property('normal',0,{c:cell_layer_surface_normals[c] for c in cell_vertex_topomesh.wisps(0)})

In [ ]:
actors = []

vertex_actor = VtkActorTopomesh(cell_vertex_topomesh,0,'layer',glyph_scale=1)
vertex_actor.update(colormap='jet',value_range=(0,4))
actors += [vertex_actor]

# seg_image_data = vtk_image_data_from_image(seg_img.get_array()%256,voxelsize=seg_img.voxelsize)
# seg_img_volume = vtk_image_volume(seg_image_data,alpha_mode='label',background_label=1,
#                                   colormap='glasbey',value_range=(0,255),opacity=1)
actors += [no_seg_img_volume]

normal_actor = VtkActorTopomesh(cell_vertex_topomesh,0,property_name='normal',glyph_scale=5.)
normal_actor.update(colormap=plain_colormap('chartreuse'))
actors += [normal_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_layer_vertex_normal.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
vtk_display_notebook(actors)

### Computing cell heights

In [ ]:
compute_tissue_cell_heights(tissue)
all_cell_heights = tissue.label.get_property('height')

In [ ]:
actors = []

height_cells = [c for c in cell_labels
                if cell_center[c][1] > 0.48*seg_img.extent[1]
                and not  np.isnan(all_cell_heights[c])]

height_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                              labels=height_cells,
                                              cell_property=all_cell_heights,
                                              cell_polydatas=cell_polydatas)
height_polydata = vtk_combine_polydatas([height_polydatas[l] for l in height_cells])
height_actor = vtk_actor(height_polydata,colormap='jet',value_range=(0,20))
actors += [height_actor]


vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_height_polydatas.png",focal_point=(0, 1, 0), view_up=(0, 0, microscope_orientation), size=(2000, 2000))
vtk_display_notebook(actors)

### Estimating L1 cell surface curvature

In [ ]:
curvature_properties = ['mean_curvature', 'gaussian_curvature', 'principal_curvature_min', 'principal_curvature_max']

for property_name in curvature_properties:
    compute_tissue_cell_property_from_surface_mesh(tissue, property_name, surface_topomesh)
    cell_vertex_topomesh.update_wisp_property(property_name,0,{c:tissue.label.get_property(property_name)[c]
                                                          for c in cell_vertex_topomesh.wisps(0)})

In [ ]:
actors = []

l1_cells = [c for c in cell_labels if cell_layer[c] == 1]

curvature_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                  labels=l1_cells,
                                                  cell_property=tissue.label.get_property('mean_curvature'),
                                                  cell_polydatas=cell_polydatas)
curvature_polydata = vtk_combine_polydatas([curvature_polydatas[l] for l in l1_cells])
curvature_actor = vtk_actor(curvature_polydata,colormap='RdBu_r',value_range=(-5e-2,5e-2))
actors += [curvature_actor]

actors += [no_seg_img_volume]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_curvature_polydatas.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))
vtk_display_notebook(actors)

### Delineating the meristem zone

In [ ]:
min_curvature_threshold = -5e-3

surface_topomesh.update_wisp_property('meristem', 2, {f:surface_topomesh.wisp_property('principal_curvature_min', 2)[f] > min_curvature_threshold for f in surface_topomesh.wisps(2)})
for iteration in range(3):
    topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 2, 'dilation', iterations=1, contour_value=0)
    topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 2, 'erosion', iterations=1, contour_value=1)
topomesh_binary_property_fill_holes(surface_topomesh, 'meristem', 2)

compute_topomesh_property(surface_topomesh,'faces',0)
vertex_face_meristem = surface_topomesh.wisp_property('meristem',2).values(surface_topomesh.wisp_property('faces',0).values(list(surface_topomesh.wisps(0))))
#vertex_meristem = list(map(np.all,vertex_face_meristem))
vertex_meristem = list(map(lambda m:int(np.mean(m)>0.5),vertex_face_meristem))
#vertex_meristem = list(map(np.any,vertex_face_meristem))

surface_topomesh.update_wisp_property('meristem',0,dict(zip(surface_topomesh.wisps(0),vertex_meristem)))
for iteration in range(3):
    topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 0, 'dilation', iterations=1, contour_value=0)
    topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 0, 'erosion', iterations=1, contour_value=1)

In [ ]:
convex_topomesh = property_filtering_sub_topomesh(surface_topomesh,'meristem',0,(1,1))
compute_topomesh_property(convex_topomesh, 'area', 2)

meristem_components = topomesh_connected_components(convex_topomesh, degree=2)

meristem_components_areas = np.array([mesh.wisp_property('area', 2).values(list(mesh.wisps(2))).sum() for mesh in meristem_components])
meristem_components_center = np.array([mesh.wisp_property('barycenter', 0).values(list(mesh.wisps(0))).mean(axis=0) for mesh in meristem_components])
meristem_components_scores = np.power(meristem_components_areas,1/2) / np.linalg.norm((meristem_components_center - np.array(seg_img.extent)/2.)[:,:2],axis=1)

meristem_topomesh = meristem_components[np.argmax(meristem_components_scores)]

surface_topomesh.update_wisp_property('meristem',0,{v:int(v in meristem_topomesh.wisps(0)) for v in surface_topomesh.wisps(0)})
#topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 0, 'dilation', iterations=1, contour_value=0)


In [ ]:
actors = []

surface_actor = VtkActorTopomesh(surface_topomesh,2,property_name='meristem',property_degree=0)
surface_actor.update(colormap='glasbey',value_range=(-1,254))
actors += [surface_actor]

actors += [no_seg_img_volume]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_meristem_mesh.png", focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))


In [ ]:
compute_tissue_cell_binary_property_from_surface_mesh(tissue, 'meristem', surface_topomesh, method='any')

In [ ]:
actors = []

meristem_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                 labels=l1_cells,
                                                 cell_property=tissue.label.get_property('meristem'),
                                                 cell_polydatas=cell_polydatas)
meristem_polydata = vtk_combine_polydatas([meristem_polydatas[l] for l in l1_cells])
meristem_actor = vtk_actor(meristem_polydata,colormap='glasbey',value_range=(-1,254))
actors += [meristem_actor]

actors += [no_seg_img_volume]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_meristem_mesh_polydatas_before.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))


In [ ]:
tissue_cell_binary_property_morphological_operation(tissue,'meristem',method='closing',iterations=3)
tissue_cell_binary_property_morphological_operation(tissue,'meristem',method='opening',iterations=3)


In [ ]:
actors = []

meristem_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                 labels=l1_cells,
                                                 cell_property=tissue.label.get_property('meristem'),
                                                 cell_polydatas=cell_polydatas)
meristem_polydata = vtk_combine_polydatas([meristem_polydatas[l] for l in l1_cells])
meristem_actor = vtk_actor(meristem_polydata,colormap='glasbey',value_range=(-1,254))
actors += [meristem_actor]

actors += [no_seg_img_volume]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_meristem_mesh_polydatas.png",focal_point=(0,0,-microscope_orientation),view_up=(0,microscope_orientation,0),size=(2000,2000))

vtk_display_notebook(actors)

In [ ]:
tissue_cell_binary_property_layer_propagation(tissue, 'meristem')

In [ ]:
actors = []

meristem_cells = [c for c in cell_labels
                  if cell_center[c][1] > 0.48*seg_img.extent[1]]

meristem_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                 labels=meristem_cells,
                                                 cell_property=tissue.label.get_property('meristem'),
                                                 cell_polydatas=cell_polydatas)
meristem_polydata = vtk_combine_polydatas([meristem_polydatas[l] for l in meristem_cells])
meristem_actor = vtk_actor(meristem_polydata, colormap='glasbey', value_range=(-1, 254))
actors += [meristem_actor]

vtk_save_screenshot_actors(actors, dirname + '/' + filename + '/' + filename + "_segmentation_cell_meristem_polydatas_all_layers.png", focal_point=(0, 1, 0), view_up=(0, 0, microscope_orientation), size=(2000, 2000))
vtk_display_notebook(actors)

### Meristematic cell height per layer

In [ ]:
layer_colors = dict(zip([1,2,3,4],['deepskyblue','chartreuse','orange','darkred']))

cell_meristem = tissue.label.get_property('meristem')

layer_height = {}
layer_height_iqr = {}

figure = plt.figure(2)
figure.clf()
for layer in [1,2,3]:
    layer_cells = np.array([l for l in cell_labels if cell_layer[l] == layer])
    layer_meristem_cells = np.array([l for l in layer_cells if cell_meristem[l]==1])
    layer_cell_heights = [all_cell_heights[l] for l in layer_meristem_cells]

    layer_height[layer] = np.nanmedian(layer_cell_heights)
    layer_height_iqr[layer] = np.nanpercentile(layer_cell_heights,75) - np.nanpercentile(layer_cell_heights,25)
    
    draw_box(figure,layer_cell_heights,box_x=layer,box_width=0.33,color=layer_colors[layer],outlier_size=5)
figure.gca().set_xlim(0,4)
figure.gca().set_xticks([1,2,3])
figure.gca().set_xticklabels(['L'+str(l) for l in [1,2,3]],size=24)
figure.gca().set_ylim(0,20)
figure.gca().set_ylabel("Layer height ($\mu$m)",size=24)
figure.set_size_inches(10,10)

figure.tight_layout()
boxplot_filename = dirname + '/' + filename + '/' + filename + "_segmentation_meristem_layer_height_boxplot.png"
figure.savefig(boxplot_filename)

### Cell data export

In [ ]:
cell_df = tissue_analysis_to_dataframe(tissue, element='cell')
data_filename = dirname + '/' + filename + '/' + filename + "_segmentation_data.csv"
cell_df.to_csv(data_filename,index=False)

### Meristem data export

In [ ]:
meristem_l1_volume = np.nansum([cell_volumes[c] for c in l1_cells if cell_meristem[c]])
meristem_area = meristem_l1_volume/layer_height[1]
meristem_radius = np.sqrt(meristem_area/np.pi)

cell_curvatures = tissue.label.get_property('mean_curvature')
meristem_curvature = np.nansum([cell_volumes[c]*cell_curvatures[c] for c in l1_cells if cell_meristem[c]])/meristem_l1_volume

In [ ]:
meristem_data = {}
meristem_data['filename'] = [filename]
meristem_data['area'] = [meristem_area]
meristem_data['diameter'] = [2*meristem_radius]
meristem_data['mean_curvature'] = [meristem_curvature]
for layer in [1,2,3]:
    meristem_data['L'+str(layer)+'_height'] = [layer_height[layer]]
    meristem_data['L'+str(layer)+'_height_interquartile'] = [layer_height_iqr[layer]]

meristem_df = pd.DataFrame().from_dict(meristem_data)

meristem_data_filename = dirname + '/' + filename + '/' + filename + '_meristem_data.csv'
meristem_df.to_csv(meristem_data_filename,index=False)
meristem_df